In [1]:
import numpy as np
from astropy.cosmology import Planck15 as cosmo
import pandas as pd

# adding path to GPR_for_IM directory in order to import relevant scripts
import sys
sys.path.append('../')
import obs_tools as obs

Frequency range of data:

In [2]:
df = 1 # frequency resolution of data (MHz)
vmin = 899 # min frequency (MHz)
vmax = 1184 # max frequency (MHz)
freqs = np.arange(vmin, vmax, df)

Load unsmoothed data:

In [3]:
gsync = np.load('../Data/dT_sync_Stripe82_noBeam.npy')
free = np.load('../Data/dT_free_Stripe82_noBeam.npy')
psource = np.load('../Data/dT_psource_Stripe82_noBeam.npy')
pleak = np.load('../Data/dT_pleak_Stripe82_noBeam.npy')
FGnopol_data = gsync + free + psource
FGwpol_data = gsync + free + psource + pleak
HI_data = np.load('../Data/T_HI-MDSAGE_z_0.39.npy')
HI_data = np.swapaxes(HI_data, 1, 2)
noise_data = np.load('../Data/dT_noise.npy')
HIFGpolnoise_data_smoothed_mock = np.load('../Data/HIFGpolnoisemock_data_smoothed.npy')
smoothed_mock = np.load('../Data/Smoothed_mock.npy')

Smooth data by telescope beam:

In [4]:
# smoothing maps
lx,ly,lz = 1000,1000, 924.78 #Mpc/h
zmax = 0.58
sigma_beam = 1.55
FGnopol_HI_noise_smoothed = obs.ConvolveCube(np.copy(FGnopol_data+HI_data+noise_data),zmax,lx,ly,sigma_beam, cosmo)
FGwpol_HI_noise_smoothed = obs.ConvolveCube(np.copy(FGwpol_data+HI_data+noise_data),zmax,lx,ly, sigma_beam, cosmo)
FGnopol_smoothed = obs.ConvolveCube(np.copy(FGnopol_data),zmax,lx,ly,sigma_beam, cosmo)
FGwpol_smoothed = obs.ConvolveCube(np.copy(FGwpol_data),zmax,lx,ly,sigma_beam, cosmo)
HI_noise_smoothed = obs.ConvolveCube(np.copy(HI_data+noise_data),zmax,lx,ly,sigma_beam, cosmo)
HI_smoothed = obs.ConvolveCube(np.copy(HI_data),zmax,lx,ly,sigma_beam, cosmo)
noise_smoothed = obs.ConvolveCube(np.copy(noise_data),zmax,lx,ly,sigma_beam, cosmo)
gsync_smoothed = obs.ConvolveCube(np.copy(gsync),zmax,lx,ly,sigma_beam, cosmo)
free_smoothed = obs.ConvolveCube(np.copy(free),zmax,lx,ly,sigma_beam, cosmo)
psource_smoothed = obs.ConvolveCube(np.copy(psource),zmax,lx,ly,sigma_beam, cosmo)
pleak_smoothed = obs.ConvolveCube(np.copy(pleak),zmax,lx,ly,sigma_beam, cosmo)

Make pickle file:

In [5]:
nobeam_fg = pd.Series({'sync':gsync, 'free':free, 'psource':psource, 'pleak':pleak})
nobeam = pd.Series({'foregrounds':nobeam_fg, 'HI':HI_data, 'noise':noise_data})

wbeam_fg = pd.Series({'sync':gsync_smoothed, 'free':free_smoothed, 'psource':psource_smoothed, 'pleak':pleak_smoothed,
                     'all_nopol':FGnopol_smoothed, 'all_wpol':FGwpol_smoothed})
wbeam = pd.Series({'foregrounds':wbeam_fg, 'HI_noise':HI_noise_smoothed, 'noise':noise_smoothed, 'HI':HI_smoothed, 
                  'FGwpol_HI_noise':FGwpol_HI_noise_smoothed, 'FGnopol_HI_noise':FGnopol_HI_noise_smoothed, 
                  'FGwpol_HI_noise_lognorm':HIFGpolnoise_data_smoothed_mock, 'lognorm':smoothed_mock})

S = pd.Series({'freqs': freqs, 'nobeam': nobeam, 'beam':wbeam})

S.to_pickle('../Data/data.pkl')